In [10]:
import torch
import math
from statistics import NormalDist

In [2]:
# convert predicted classes to one hot ecoding
# define distribution function
# calculate mean and std for quantitative predictors and probability for qualitative predictors
# multiply the probabilities for each predictor with individual class probability
# assign the class with max prob

In [12]:
def gaussian_func(mean,std,x):
  return math.exp(-(1/2)*((x-mean)/std)**2)/ (std*2*math.pi)

NormalDist(mu=45,sigma = 5).pdf(30)

0.0008863696823876014